In [2]:
# 🧩 Cell 1 — TCS.NS, 1-minute data, robust mapping
from datetime import datetime, timedelta
import yfinance as yf, pandas as pd, pandas_ta as ta

# 1) Download last 7 days of 1-minute candles
symbol = "TCS.NS"
end, start = datetime.now(), datetime.now() - timedelta(days=7)
df = yf.download(symbol, start=start.strftime("%Y-%m-%d"),
                 end=end.strftime("%Y-%m-%d"),
                 interval="1m", auto_adjust=False, progress=False)

# 2) Flatten any MultiIndex and clean names
if hasattr(df.columns, "to_flat_index"):
    df.columns = df.columns.to_flat_index()
df.columns = [
    "_".join(map(str, c)) if isinstance(c, tuple) else str(c)
    for c in df.columns
]

# 3) Add datetime string column
df = df.reset_index()
df["time"] = pd.to_datetime(df["Datetime"]).dt.strftime("%Y-%m-%d %H:%M")

# 4) Map actual OHLCV column names (handles ticker suffix)
def map_ohlcv(frame):
    out = {}
    for base in ["Open", "High", "Low", "Close", "Volume"]:
        out[base] = next(c for c in frame.columns if base.lower() in c.lower())
    return out
ohlcv = map_ohlcv(df)

# 5) Technical indicators
df.ta.sma(length=20,  append=True)
df.ta.ema(length=12,  append=True)
df.ta.ema(length=26,  append=True)
df.ta.macd(append=True)
df.ta.rsi(length=14,  append=True)
df.ta.bbands(length=20, append=True)
df.ta.atr(length=14,  append=True)
df.ta.stoch(append=True)
df.ta.adx(length=14,  append=True)
df.ta.willr(length=14, append=True)
df["OBV"]  = ta.obv( df[ohlcv["Close"]], df[ohlcv["Volume"]] )
df["VWAP"] = ta.vwap(df[ohlcv["High"]], df[ohlcv["Low"]],
                     df[ohlcv["Close"]], df[ohlcv["Volume"]])
df.ta.sqzpro(append=True)

print(f"Downloaded {len(df)} 1-minute rows for {symbol}")
print(df.head())


[!] VWAP requires an ordered DatetimeIndex.


AttributeError: 'AnalysisIndicators' object has no attribute 'sqzpro'